In [1]:
from __future__ import division

In [2]:
!pip install pymupdf
!pip install tqdm
!pip install python-docx
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.0 MB 261 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.8 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=b4e884a022c2f5c052308286c187e836e736102a2a1ab42df04aa07fef78ddb0
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 4.8 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully unins

In [3]:
import os
from pathlib import Path
import string
import nltk
nltk.download('stopwords')
import docx
import pytesseract
from PIL import Image
# https://stackoverflow.com/questions/22676/how-to-download-a-file-over-http
import urllib.request
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
# https://stackoverflow.com/questions/2693820/extract-images-from-pdf-without-resampling-in-python
import fitz  # pip install --upgrade pip; pip install --upgrade pymupdf
from tqdm import tqdm # pip install tqdm

from typing_extensions import ParamSpecArgs
import fitz # install using: pip install PyMuPDF
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import shutil

from struct import pack, unpack

import binascii
import heapq

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1PhVWY5J3XqMYglbyZXzgINWU7S7AvDb_?usp=sharing
folder_dataset_cranfield = "Cranfield Dataset"

Retrieving folder list
Processing file 1CvoJEpZjw3s4tzHVGE8vn79L9iLlErYh cran.all.1400
Processing file 16vwKAHUoNCIg3kVq_ASJm7FNk7uh_5tI cran.qry
Processing file 1dprbwLD-c4thBjzJM8TEPGZ7d1UP6Tyh cranqrel
Processing file 1fISYvFErzn8aYz2zk1-BSalPs60MOhHk cranqrel.readme
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1CvoJEpZjw3s4tzHVGE8vn79L9iLlErYh
To: /content/Cranfield Dataset/cran.all.1400
100% 1.64M/1.64M [00:00<00:00, 80.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=16vwKAHUoNCIg3kVq_ASJm7FNk7uh_5tI
To: /content/Cranfield Dataset/cran.qry
100% 29.0k/29.0k [00:00<00:00, 33.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dprbwLD-c4thBjzJM8TEPGZ7d1UP6Tyh
To: /content/Cranfield Dataset/cranqrel
100% 19.5k/19.5k [00:00<00:00, 26.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fISYvFErzn8aYz2zk1-BSalPs60MOhHk
To: /content/Cranfield Dataset/cranq

In [5]:
# {query_id : [document_id]}
cranqrel = {}
filename_cranqrel = "cranqrel"
file_path_cranqrel = os.path.join(folder_dataset_cranfield, filename_cranqrel)
with open(file_path_cranqrel) as f:
  for line in f.readlines():
    line = line.rstrip()
    cranqrel_line = line.rstrip().split()
    query_id = int(cranqrel_line[0])
    doc_id = int(cranqrel_line[1])
    if query_id not in cranqrel:
      cranqrel[query_id] = []
    cranqrel[query_id].append(doc_id)

In [6]:
print(cranqrel)

{1: [184, 29, 31, 12, 51, 102, 13, 14, 15, 57, 378, 859, 185, 30, 37, 52, 142, 195, 875, 56, 66, 95, 462, 497, 858, 876, 879, 880, 486], 2: [12, 15, 184, 858, 51, 102, 202, 14, 52, 380, 746, 859, 948, 285, 390, 391, 442, 497, 643, 856, 857, 877, 864, 658, 486], 3: [5, 6, 90, 91, 119, 144, 181, 399, 485], 4: [236, 166, 488], 5: [552, 401, 1297, 1296, 488], 6: [99, 115, 257, 258, 491], 7: [20, 56, 57, 58, 19, 492], 8: [48, 122, 20, 58, 196, 354, 360, 197, 999, 1112, 1005, 492], 9: [21, 22, 550, 534], 10: [259, 405, 302, 436, 437, 438, 998, 1011, 493], 11: [27, 28, 262, 160, 20, 263, 654, 495], 12: [86, 194, 650, 649, 652, 624], 13: [64, 265, 65, 311, 496], 14: [64, 65, 496], 15: [463, 462, 497], 16: [266, 106, 196, 498], 17: [106, 196, 498], 18: [196, 197, 198, 498], 19: [32, 67, 164, 639, 715, 716, 719, 1379, 717, 499], 20: [87, 88, 104, 267, 268, 269, 270, 407, 408, 500], 21: [271, 16, 413, 414, 502], 22: [68, 502], 23: [900, 902, 200, 201, 601, 899, 903, 593, 199, 594, 901, 544, 597, 

In [7]:
# {query_id : query}
cran_qry = {}
filename_cran_qry = "cran.qry"
file_path_cran_qry = os.path.join(folder_dataset_cranfield, filename_cran_qry)
with open(file_path_cran_qry) as f:
  searching_w = False
  reading_w = False
  current_query_id = 0
  query_to_be_merged = []
  for line in f.readlines():
    line = line.rstrip()
    line_result = line.rstrip().split()
    if not searching_w and not reading_w:
      # initialization
      if line_result[0] == ".I":
        searching_w = True
        current_query_id += 1
    elif searching_w:
      if line_result[0] == ".W":
        searching_w = False
        reading_w = True
    
    # searching_w = False
    # reading_w = True
    else:
      if line_result[0] != ".I":
        query_to_be_merged.append(line.rstrip())
      else:
        cran_qry[current_query_id] = " ".join(query_to_be_merged)
        query_to_be_merged = []
        searching_w = True
        reading_w = False
        current_query_id += 1
  # last query
  cran_qry[current_query_id] = " ".join(query_to_be_merged)

In [8]:
print(cran_qry)

{1: 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .', 2: 'what are the structural and aeroelastic problems associated with flight of high speed aircraft .', 3: 'what problems of heat conduction in composite slabs have been solved so far .', 4: 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium .', 5: 'what chemical kinetic system is applicable to hypersonic aerodynamic problems .', 6: 'what theoretical and experimental guides do we have as to turbulent couette flow behaviour .', 7: 'is it possible to relate the available pressure distributions for an ogive forebody at zero angle of attack to the lower surface pressures of an equivalent ogive forebody at angle of attack .', 8: 'what methods -dash exact or approximate -dash are presently available for predicting body pressures at angle 

In [9]:
# {doc_id : [title, author, publisher, content]}
cran_doc = {}
filename_cran_doc = "cran.all.1400"
file_path_cran_doc = os.path.join(folder_dataset_cranfield, filename_cran_doc)
with open(file_path_cran_doc) as f:
  str_to_be_merged = []
  title = None
  author = None
  publisher = None
  searching_t = False
  searching_a = False
  searching_b = False
  searching_w = False
  reading_w = False
  current_doc_id = 0
  for line in f.readlines():
    line = line.rstrip()
    if not searching_t and not searching_a and not searching_b and not searching_w and not reading_w:
      # initialization
      if line.split()[0] == ".I":
        searching_t = True
        searching_a = False
        searching_b = False
        searching_w = False
        reading_w = False
        current_doc_id += 1
    elif searching_t:
      if line.split()[0] == ".T":
        searching_t = False
        searching_a = True
        searching_b = False
        searching_w = False
        reading_w = False
    elif searching_a:
      if line.split()[0] != ".A":
        str_to_be_merged.append(line)
      else:
        searching_t = False
        searching_a = False
        searching_b = True
        searching_w = False
        reading_w = False
        title = " ".join(str_to_be_merged)
        str_to_be_merged = []
    elif searching_b:
      if line.split()[0] != ".B":
        str_to_be_merged.append(line)
      else:
        searching_t = False
        searching_a = False
        searching_b = False
        searching_w = True
        reading_w = False
        author = " ".join(str_to_be_merged)
        str_to_be_merged = []
    elif searching_w:
      if line.split()[0] != ".W":
        str_to_be_merged.append(line)
      else:
        searching_t = False
        searching_a = False
        searching_b = False
        searching_w = False
        reading_w = True
        publisher = " ".join(str_to_be_merged)
        str_to_be_merged = []
    else:
      if line.split()[0] != ".I":
        str_to_be_merged.append(line)
      else:
        searching_t = True
        searching_a = False
        searching_b = False
        searching_w = False
        reading_w = False
        document_content = " ".join(str_to_be_merged)
        str_to_be_merged = []
        cran_doc[current_doc_id] = [title, author, publisher, document_content]
        title = None
        author = None
        publisher = None
        document_content = None
        current_doc_id += 1
  # last doc
  document_content = " ".join(str_to_be_merged)
  cran_doc[current_doc_id] = [title, author, publisher, document_content]
  title = None
  author = None
  publisher = None
  document_content = None

In [10]:
print(len(cran_doc))

1400


In [11]:
print(cran_doc[1])

['experimental investigation of the aerodynamics of a wing in a slipstream .', 'brenckman,m.', 'j. ae. scs. 25, 1958, 324.', 'experimental investigation of the aerodynamics of a wing in a slipstream .   an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem .   the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary-layer-control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory .   an empirical evaluation of the destalling effects was made for the specif

In [12]:
folder_dokumen_cranfield = "dokumen_cranfield"
if not os.path.exists(folder_dokumen_cranfield) and not os.path.isfile(folder_dokumen_cranfield):
    # create new folder
    os.makedirs(folder_dokumen_cranfield)
# dokumen yang dipakai hanya dokumen 1 sampai 50
# for i in range(50):
#   doc_id = i+1
for doc_id, doc_data in cran_doc.items():
  doc_id = str(doc_id)
  # doc_data = cran_doc[doc_id]
  title = doc_data[0]
  author = doc_data[1]
  publisher = doc_data[2]
  content = doc_data[3]
  doc_path = os.path.join(folder_dokumen_cranfield, "%s.txt" % doc_id)
  with open(doc_path, "w+") as f: pass
  f = open(doc_path, "a")
  f.write(title)
  f.write("\n")
  f.write(author)
  f.write("\n")
  f.write(publisher)
  f.write("\n")
  f.write(content)
  f.close()

In [13]:
workspace_folder_path = None
while True:
  print("Mohon masukkan alamat folder yang dapat digunakan oleh program ini " + \
      "untuk beroperasi: ")
  # Folder yang digunakan oleh software untuk beroperasi
  workspace_folder_path = str(input())
  if not os.path.exists(workspace_folder_path):
    print("Mohon maaf, folder tidak tersedia atau alamat folder tidak valid, mohon masukkan kembali alamat folder")
  elif not os.path.isdir(workspace_folder_path):
    print("Mohon maaf, alamat folder tidak valid, mohon masukkan kembali alamat folder")
  else:
    print("Alamat folder berhasil disimpan")
    break

Mohon masukkan alamat folder yang dapat digunakan oleh program ini untuk beroperasi: 
Indexing
Alamat folder berhasil disimpan


In [14]:
indexing_folder = os.path.join(workspace_folder_path, "indexing_folder")
indexing_file = os.path.join(indexing_folder, "index.txt")
indexing_document_path_to_document_id_file = os.path.join(indexing_folder, "document_id.txt")
indexing_ulang = None

if os.path.exists(indexing_file) and os.path.exists(indexing_document_path_to_document_id_file):
  while True:
    print("Telah ditemukan hasil indexing sebelumnya. Ketik Y untuk" + " " +
          "menggunakan hasil indexing sebelumnya, ketik N untuk" + " " +
          "melakukan indexing ulang. Jika melakukan indexing ulang" + " " + 
          "maka hasil indexing sebelumnya akan dihapus.")
    use_previous_indexing = str(input()).lower()
    if len(use_previous_indexing) > 1 or len(use_previous_indexing) < 1 or \
    (use_previous_indexing.lower() != "y" and \
    use_previous_indexing.lower() != "n"):
      print("Mohon ketik Y atau N.")
    elif use_previous_indexing.lower() == "y":
      while True:
        print("Apakah anda yakin ingin menggunakan hasil indexing sebelumnya?"+
              " " + "(Y/N)")
        use_previous_indexing = str(input())
        if len(use_previous_indexing) > 1 or len(use_previous_indexing) < 1 or \
        (use_previous_indexing.lower() != "y" and \
        use_previous_indexing.lower() != "n"):
          print("Mohon ketik Y atau N.")
        elif use_previous_indexing.lower() == "y":
          print("Akan menggunakan hasil indexing sebelumnya.")
          indexing_ulang = False
          break
        elif use_previous_indexing.lower() == "n":
          break
        else:
          raise ValueError("Telah terjadi error.")
    elif use_previous_indexing.lower() == "n":
      while True:
        print("Apakah anda yakin ingin melakukan indexing ulang dan"+
              " " + "menghapus hasil indexing sebelumya?" + " " +
              "(Y/N)")
        use_previous_indexing = str(input())
        if len(use_previous_indexing) > 1 or len(use_previous_indexing) < 1 or \
        (use_previous_indexing.lower() != "y" and \
        use_previous_indexing.lower() != "n"):
          print("Mohon ketik Y atau N.")
        elif use_previous_indexing.lower() == "y":
          print("Akan melakukan indexing ulang dan menghapus hasil indexing"+
                " " + "sebelumnya")
          indexing_ulang = True
          break
        elif use_previous_indexing.lower() == "n":
          break
        else:
          raise ValueError("Telah terjadi error.")
    else:
      raise ValueError("Telah terjadi error")
    if indexing_ulang is not None:
      break
else:
  indexing_ulang = True


In [15]:
def text_transformation(word):
  ps = PorterStemmer()
  stop_words = stopwords.words('english')
  # remove punctuation
  word = word.translate(str.maketrans('', '', string.punctuation))
  # stopword removal
  if word in stop_words:
    word = ''
  else:
    word = ps.stem(word)
  return word

In [16]:
print(text_transformation("propeller"))

propel


In [17]:
# SPIMI

# terms adalah array 2 dimensi
# terms[0] adalah string
# terms[1] adalah integer

output_directory = indexing_folder

block_prefix = "block"
block_suffix = ".txt"

output_index = indexing_file
output_document_id = indexing_document_path_to_document_id_file

dictionary = {}
# Jika value nya di set maka akan menulis block ke file secara otomatis jika
# dictionary sudah penuh
dictionary_max_key = [10000]
dictionary_max_item = [50000]
dictionary_key_count = [0]
dictionary_item_count = [0]
highest_document_id = [0]
document_path_to_document_id = {}
block_files = []
block_number = [0]

# fungsi yang diawali dengan underscore "_" adalah fungsi privat
# artinya tidak boleh dipanggil oleh proses luar

def _add_to_dictionary(dictionary, term, dictionary_key_count=dictionary_key_count):
  if term not in dictionary:
    dictionary_key_count[0] += 1
    dictionary[term] = []
  return dictionary[term]

def _get_postings_list(dictionary, term):
  return dictionary[term]

def _add_to_postings_list(postings_list, document_path, dictionary_item_count=dictionary_item_count):
  dictionary_item_count[0] += 1
  postings_list.append(document_path)

def _sort_terms(dictionary):
  return [term for term in sorted(dictionary)]

def write_block_to_output_directory(block_number=block_number, \
                                    dictionary=dictionary, \
                                    block_prefix=block_prefix, \
                                    block_suffix=block_suffix, \
                                    dictionary_key_count=dictionary_key_count, \
                                    dictionary_item_count=dictionary_item_count, \
                                    output_directory=output_directory, \
                                    document_path_to_document_id=document_path_to_document_id):
  block_number[0] += 1
  sorted_terms = _sort_terms(dictionary)

  block_file = os.path.join(output_directory, "".join([block_prefix, 
                                                        str(block_number[0]), 
                                                        block_suffix]))
  with open(block_file, 'w') as file:
      for term in sorted_terms:
          line = "%s %s\n" % (term, ' '.join([str(document_id) for document_id in dictionary[term]]))
          file.write(line)
  block_files.append(block_file)
  # reset dictionary setelah menulis ke file block
  dictionary = {}
  dictionary_key_count[0] = 0
  dictionary_item_count[0] = 0

# token adalah array-like dengan 2 data
# token[0] adalah term nya
# token[1] adalah document path
def add_terms_to_block(token, dictionary_max_key=dictionary_max_key, \
                       dictionary_max_item=dictionary_max_item, \
                       dictionary_key_count=dictionary_key_count, \
                       dictionary_item_count=dictionary_item_count, \
                       highest_document_id=highest_document_id, \
                       document_path_to_document_id=document_path_to_document_id):
    try:
      if token[0] and token[1]:
        pass
    except:
      raise ValueError("Error: token tidak berbentuk array, namun %s" % \
                       type(token))
    if len(token) < 2:
      raise ValueError("Error: jumlah elemen pada token kurang dari 2")
    if not os.path.exists(token[1]):
      raise ValueError("Error: file dokumen pada token[1] tidak ditemukan ")
    if not os.path.isfile(token[1]):
      raise ValueError("Error: path pada token[1] mengarah ke folder" + " "+
                       "bukan file")


    if dictionary_max_key is not None and dictionary_max_item is not None:
      if dictionary_max_key[0] <= 0 or dictionary_max_item[0] <= 0:
        raise ValueError("Error: dictionary_max_key dan dictionary_max_item"+
                         " " + "tidak boleh kurang dari atau sama dengan 0")
      # jika ukuran dictionary melebihi batas, maka tulis dictionary ke file
      if dictionary_key_count[0] >= dictionary_max_key[0] or \
      dictionary_item_count[0] >= dictionary_max_item[0]:
        write_block_to_output_directory()

    if token[1] not in document_path_to_document_id:
      highest_document_id[0] += 1
      document_path_to_document_id[token[1]] = highest_document_id[0]

    terms = [token[0], document_path_to_document_id[token[1]]]
    if token[0] not in dictionary:  
      postings_list = _add_to_dictionary(dictionary, terms[0])
    else:
      postings_list = _get_postings_list(dictionary, terms[0])
    _add_to_postings_list(postings_list, terms[1])

def construct_index():
  write_block_to_output_directory()
  return _merge_blocks(block_files)

def _merge_blocks(block_files, output_index=output_index, document_path_to_document_id=document_path_to_document_id):
  block_files = [open(block_file) for block_file in block_files]
  lines = [block_file.readline()[:-1] for block_file in block_files]
  most_recent_term = ""

  index = 0
  for block_file in block_files:
      if lines[index] == "":
          block_files.pop(index)
          lines.pop(index)
      else:
          index += 1

  with open(output_index, "w") as output_index:
      while len(block_files) > 0:

          min_index = lines.index(min(lines))
          line = lines[min_index]
          current_term = line.split()[0]
          current_postings = " ".join(map(str, sorted(list(map(int, line.split()[1:])))))

          if current_term != most_recent_term:
              output_index.write("\n%s %s" % (current_term, current_postings))
              most_recent_term = current_term
          else:
              output_index.write(" %s" % current_postings)

          lines[min_index] = block_files[min_index].readline()[:-1]

          if lines[min_index] == "":
              block_files[min_index].close()
              block_files.pop(min_index)
              lines.pop(min_index)

      output_index.close()
  with open(indexing_document_path_to_document_id_file, 'w') as file:
      for document_path in document_path_to_document_id:
          line1 = "%s\n" % document_path
          line2 = "%s\n" % str(document_path_to_document_id[document_path])
          file.write(line1)
          file.write(line2)
  file.close()

  return get_index()

def get_index(index_file=indexing_file, \
               document_id_file=indexing_document_path_to_document_id_file):
  inverted_index = {}
  document_id_and_path = {}

  index_file = open(output_index)
  index_file.readline()

  for line in index_file:
      line = line.split()
      inverted_index[line[0]] = sorted(map(int, (line[1:])))

  document_id_file = open(document_id_file)
  line0 = None
  line1 = None
  for line in document_id_file:
      # line[0] = document path
      # line[1] = document id
      # document id => document path
      if line0 is None:
        line0 = line
        line1 = None
      elif line1 is None:
        line1 = line
        document_id_and_path[int(line1)] = line0.rstrip()
        line0 = None
        line1 = None
  document_id_file.close()
  
  return (inverted_index, document_id_and_path)

In [18]:
def send_text_to_spimi(list_of_text, file_path):
    for i in range(len(list_of_text)):
      text_to_process = list_of_text[i].split()
      # Text Transformation
      for j in range(len(text_to_process)):
        word_to_process = text_to_process[j]

        word_to_process = text_transformation(word_to_process)
        if word_to_process != '':
          add_terms_to_block((word_to_process, file_path))
  
def scrape_file(file_path):
  if not os.path.exists(file_path) and not os.path.isdir(file_path):
        raise ValueError("File does not exist: %s" % str(file_path))
  file_extension = Path(file_path).suffix
  if file_extension == ".png" or file_extension == ".jpg" or file_extension == ".jpeg":
      text_image = pytesseract.image_to_string(Image.open(file_path))
      send_text_to_spimi([text_image], file_path)
  elif file_extension == ".txt":
    with open(file_path) as f:
      for line in f.readlines():
        send_text_to_spimi([line], file_path)
    f.close()
  elif file_extension == ".doc":
    # Belum implementasi
    pass
  elif file_extension == ".docx":
    # Belum implementasi OCR
    doc = docx.Document(file_path)
    for para in doc.paragraphs:
      send_text_to_spimi([para.text], file_path)
  elif file_extension == ".pdf":
    # Belum Implementasi OCR
    with fitz.open(file_path) as doc:
        for page in doc:
            send_text_to_spimi([page.get_text("text")], file_path)
  elif file_extension == ".sgm":
    # Belum Implementasi OCR
    with open(file_path) as f:
      for line in f.readlines():
        send_text_to_spimi([line], file_path)
  else:
    # skip karena format file tidak disupport
    pass

In [19]:
if indexing_ulang:
  list_of_path = []
  if not os.path.exists(indexing_folder):
    # buat folder
    os.makedirs(indexing_folder)
  else:
    # hapus folder dan isinya lalu buat foldernya lagi
    shutil.rmtree(indexing_folder)
    os.makedirs(indexing_folder)

  while True:
    print("Masukkan alamat folder/file yang ingin dimasukkan ke mesin pencari")
    path_to_append = input()
    if not os.path.exists(path_to_append):
      print("Alamat folder/file tidak ditemukan, mohon masukkan kembali" +
            " " + "alamat folder/file")
    else:
      print("Alamat folder/file berhasil disimpan")
      list_of_path.append(path_to_append)
      break

  continue_adding_path = True
  while continue_adding_path:
    while True:
      print("Apakah anda ingin memasukkan alamat folder/file lain yang ingin"+
          " " + "ditambah ke mesin pencari? (Y/N)")
      add_more_file_path = str(input()).lower()
      if len(add_more_file_path) > 1 or len(add_more_file_path) < 1 or \
      (add_more_file_path.lower() != "y" and \
      add_more_file_path.lower() != "n"):
        print("Mohon ketik Y atau N.")
      elif add_more_file_path.lower() == "y":
        continue_adding_path = True
        break
      elif add_more_file_path.lower() == "n":
        continue_adding_path = False
        break
        
    if not continue_adding_path:
      break
    print("Masukkan alamat folder/file yang ingin dimasukkan ke mesin pencari")
    path_to_append = input()
    if not os.path.exists(path_to_append):
      print("Alamat folder/file tidak ditemukan, mohon masukkan kembali" +
            " " + "alamat folder/file")
    else:
      print("Alamat folder/file berhasil disimpan")
      list_of_path.append(path_to_append)
  
  print("Sedang melakukan proses indexing, harap tunggu...")
  
  current_layer_recursion_stack = list_of_path
  next_layer_recursion_stack = []

  while len(current_layer_recursion_stack) > 0:
    path_to_process = current_layer_recursion_stack.pop()
    if not os.path.exists(path_to_process):
      print("Path does not exists, skipping path \"%s\"" % str(path_to_process))
    elif os.path.isfile(path_to_process):
      print("Scrapping file \"%s\"" % str(path_to_process))
      scrape_file(path_to_process)
    # directory
    else:
      current_folder = path_to_process
      for name in os.listdir(current_folder):
        new_path = os.path.join(current_folder, name)
        if os.path.isfile(new_path):
          print("Scrapping file \"%s\"" % str(new_path))
          scrape_file(new_path)
        else:
          next_layer_recursion_stack.append(new_path)
    if len(current_layer_recursion_stack) <= 0:
        current_layer_recursion_stack = next_layer_recursion_stack
        next_layer_recursion_stack = []
  construct_index()
  print("Proses indexing selesai!")



Masukkan alamat folder/file yang ingin dimasukkan ke mesin pencari
dokumen_cranfield
Alamat folder/file berhasil disimpan
Apakah anda ingin memasukkan alamat folder/file lain yang ingin ditambah ke mesin pencari? (Y/N)
y
Masukkan alamat folder/file yang ingin dimasukkan ke mesin pencari

Alamat folder/file tidak ditemukan, mohon masukkan kembali alamat folder/file
Apakah anda ingin memasukkan alamat folder/file lain yang ingin ditambah ke mesin pencari? (Y/N)
n
Sedang melakukan proses indexing, harap tunggu...
Scrapping file "dokumen_cranfield/645.txt"
Scrapping file "dokumen_cranfield/1149.txt"
Scrapping file "dokumen_cranfield/863.txt"
Scrapping file "dokumen_cranfield/843.txt"
Scrapping file "dokumen_cranfield/1072.txt"
Scrapping file "dokumen_cranfield/1222.txt"
Scrapping file "dokumen_cranfield/167.txt"
Scrapping file "dokumen_cranfield/109.txt"
Scrapping file "dokumen_cranfield/356.txt"
Scrapping file "dokumen_cranfield/26.txt"
Scrapping file "dokumen_cranfield/287.txt"
Scrapping

In [20]:
import math
from collections import Counter, defaultdict

def unique(x):
  list_set = set(x)
  unique_list = (list(list_set))
  return unique_list

def get_idf(index_dict, N):
  idf_dict=defaultdict(lambda: 0)
  for key, value in index_dict.items():
        unique_value = unique(value)
        idf_dict[key] = (math.log((1+N)/(len(unique_value)+1)))
  return idf_dict

def Term_frequency(index_dict, unique_word, docid):
  term_frequency = defaultdict(lambda: 0)
  for word in unique_word :
    if word in index_dict:
      posting_list = index_dict[word]
      for L in posting_list:
        if L == docid :
          term_frequency[word] += 1
    else :
      term_frequency[word] = 0
  return term_frequency

def tf_query(term, unique_word):
  term_frequency = defaultdict(lambda: 0)
  for word in unique_word :
    for i in range(len(term)):
      if word == term[i]:
        term_frequency[word] += 1
  return term_frequency

def tf_idf(tf, idf):
  tf_idf = (math.log(1+tf))*idf
  return tf_idf

def cosine(index_result, query_corpus, idf, threshold=0.001):
  tf_idf_document = defaultdict(lambda: 0)
  tf_idf_query = defaultdict(lambda: 0)
  score = {}
  query_unique_terms = unique(query_corpus)  
  query_tf = tf_query(query_corpus, query_unique_terms)
  
  for key in index_result[1].keys():
    product = []
    
    document_tf = Term_frequency(index_result[0], query_unique_terms, key)
    
    for w in query_unique_terms:
      weight_document = tf_idf(document_tf[w], idf[w])
      weight_query = tf_idf(query_tf[w], idf[w])
      product.append(weight_query*weight_document)
    if sum(product)/len(index_result[1]) > threshold:
      score[key] = sum(product)/len(index_result[1])
  return score

In [21]:
print("Sedang loading hasil indexing, harap tunggu...")
indexing_result = get_index()
print("Hasil indexing berhasil di load!")

Sedang loading hasil indexing, harap tunggu...
Hasil indexing berhasil di load!


In [22]:
print(indexing_result[0]['problem'])

[3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 12, 12, 12, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 31, 31, 31, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 45, 45, 45, 60, 60, 60, 60, 60, 60, 60, 60, 60, 67, 67, 67, 68, 68, 68, 68, 68, 68, 68, 68, 68, 69, 69, 69, 69, 69, 69, 74, 74, 74, 81, 81, 81, 81, 81, 81, 81, 81, 81, 84, 84, 84, 88, 88, 88, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 90, 90, 90, 99, 99, 99, 100, 100, 100, 103, 103, 103, 110, 110, 110, 112, 112, 112, 112, 112, 112, 114, 114, 114, 114, 114, 114, 116, 116, 116, 116, 116, 116, 116, 116, 116, 125, 125, 125, 125, 125, 125, 127, 127, 127, 127, 127, 127, 139, 139, 139, 141, 141, 141, 143, 143, 143, 143, 143, 143, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 145, 145, 145, 145, 145, 145, 146, 146, 146, 148, 148, 148, 149, 149, 149, 149, 149, 149, 154, 154, 154, 154, 154, 154, 154, 154, 154, 162, 162, 162, 175, 175, 175, 175

In [23]:
idf = get_idf(indexing_result[0], len(indexing_result[1]))
print(idf)

defaultdict(<function get_idf.<locals>.<lambda> at 0x7f05cd2ba680>, {'0': 3.0240052694893746, '0000': 6.550365794105536, '00001': 6.144900685997372, '0001': 6.144900685997372, '0002': 6.550365794105536, '0003': 6.550365794105536, '0004': 6.550365794105536, '000675': 6.550365794105536, '001': 6.144900685997372, '0010': 6.550365794105536, '0012': 6.550365794105536, '0014': 6.550365794105536, '002': 5.857218613545591, '002025': 6.550365794105536, '0025': 6.550365794105536, '002in': 6.550365794105536, '003': 6.550365794105536, '004': 6.550365794105536, '005': 5.451753505437426, '006': 6.550365794105536, '0066': 6.550365794105536, '008': 6.550365794105536, '00904': 6.550365794105536, '01': 4.940927881671436, '010': 5.634075062231381, '011': 6.550365794105536, '0117': 6.550365794105536, '012': 5.857218613545591, '0120': 6.550365794105536, '013': 6.550365794105536, '014': 6.550365794105536, '01428': 6.550365794105536, '014x10': 6.550365794105536, '015': 6.144900685997372, '015x106': 6.5503657

In [24]:
def search(indexing_result, query, idf):
  query = query.split()
  new_query = []
  for i in range(len(query)):
    if text_transformation(query[i]) != '':
      new_query.append(text_transformation(query[i]))
  query = new_query
  ranking = cosine(indexing_result, query, idf)
  return ranking

In [25]:
# indexing_result[1][doc_id_pada_index] = file_path

In [26]:
while True:
  print("Masukkan query untuk mencari dokumen, gunakan spasi untuk memisahkan kata")
  query = str(input()).lower()
  continue_search_this_query = False
  # view top k document
  top_k = 10
  while True:
    print("Apakah anda yakin ingin mencari dokumen dengan query diatas? (Y/N)")
    search_this_query = str(input()).lower()
    if len(search_this_query) > 1 or len(search_this_query) < 1 or \
    (search_this_query.lower() != "y" and \
    search_this_query.lower() != "n"):
      print("Mohon ketik Y atau N.")
    elif search_this_query.lower() == "y":
      continue_search_this_query = True
      break
    elif search_this_query.lower() == "n":
      continue_search_this_query = False
      break
  if not continue_search_this_query:
    print("Pencarian dibatalkan, silahkan masukkan query yang baru")
    continue
  else:
    print("Sedang melakukan pencarian, harap tunggu...")
    ranking = search(indexing_result, query, idf)
    if len(ranking) < top_k:
      top_k = len(ranking)
    search_result = heapq.nlargest(top_k, ranking, key=ranking.get)
    print("Ditemukan %s hasil pencarian" % len(ranking))
    print("Menampilkan %s pencarian teratas..." % top_k)
    for i in range(len(search_result)):
      print("%s. %s" % ((i+1), indexing_result[1][search_result[i]]))

Masukkan query untuk mencari dokumen, gunakan spasi untuk memisahkan kata
supersonic
Apakah anda yakin ingin mencari dokumen dengan query diatas? (Y/N)
y
Sedang melakukan pencarian, harap tunggu...
Ditemukan 237 hasil pencarian
Menampilkan 10 pencarian teratas...
1. dokumen_cranfield/216.txt
2. dokumen_cranfield/124.txt
3. dokumen_cranfield/993.txt
4. dokumen_cranfield/798.txt
5. dokumen_cranfield/80.txt
6. dokumen_cranfield/426.txt
7. dokumen_cranfield/1212.txt
8. dokumen_cranfield/200.txt
9. dokumen_cranfield/410.txt
10. dokumen_cranfield/221.txt
Masukkan query untuk mencari dokumen, gunakan spasi untuk memisahkan kata
chemical
Apakah anda yakin ingin mencari dokumen dengan query diatas? (Y/N)
y
Sedang melakukan pencarian, harap tunggu...
Ditemukan 40 hasil pencarian
Menampilkan 10 pencarian teratas...
1. dokumen_cranfield/103.txt
2. dokumen_cranfield/410.txt
3. dokumen_cranfield/1061.txt
4. dokumen_cranfield/552.txt
5. dokumen_cranfield/1296.txt
6. dokumen_cranfield/1374.txt
7. doku

KeyboardInterrupt: ignored